In [12]:
import pandas as pd
import numpy as np

def process_metric_tables(metric_tables):
    def parse_tables_manual(metric_tables):
        dataframes = []
        for table in metric_tables:
            lines = table.strip().split('\n')
            headers = lines[0].split(maxsplit=2)
            data = []
            for line in lines[1:]:
                index_space = line.index(' ')
                first_split = line[:index_space].strip()
                remaining = line[index_space:].strip()
                parts = remaining.rsplit(maxsplit=2)
                if len(parts) == 3:
                    data.append([parts[0], parts[1], parts[2]])
                else:
                    data.append([parts[0], np.nan, np.nan])
            df = pd.DataFrame(data, columns=headers)
            df[headers[1]] = pd.to_numeric(df[headers[1]], errors='coerce')
            df[headers[2]] = pd.to_numeric(df[headers[2]], errors='coerce')
            dataframes.append(df)
        return dataframes

    def calculate_statistics_ordered(dataframes):
        metrics = dataframes[0]['Metric']
        result_list = []
        for metric in metrics:
            metric_data = [df[df['Metric'] == metric] for df in dataframes]
            metric_stats = {}
            for col in ['Training', 'Validation']:
                valid_data = pd.concat([md[col] for md in metric_data]).dropna()
                if not valid_data.empty:
                    mean_val = valid_data.mean()
                    std_val = valid_data.std()
                    metric_stats[col] = (mean_val, std_val)
                else:
                    metric_stats[col] = np.nan
            result_list.append((metric, metric_stats))
        return result_list

    def format_output_ordered(statistics_ordered):
        output_df = pd.DataFrame(columns=['Metric', 'Training', 'Validation'])
        for metric, values in statistics_ordered:
            row = {'Metric': metric}
            for col in ['Training', 'Validation']:
                if col in values and not pd.isna(values[col]):
                    row[col] = f"({values[col][0]:.3f}, {values[col][1]:.3f})"
                else:
                    row[col] = np.nan
            output_df = pd.concat([output_df, pd.DataFrame([row])], ignore_index=True)
        output_string = output_df.to_string(index=False)
        return output_string

    # Process the input tables
    dataframes = parse_tables_manual(metric_tables)
    statistics = calculate_statistics_ordered(dataframes)
    output_table = format_output_ordered(statistics)
    return output_table
# Example usage
metric_tables = [
    """                              Metric  Training  Validation
0                   Average Accuracy     0.377       0.396
1     Standard Deviation of Accuracy     0.047       0.081
2                      Best Accuracy     0.495       0.692
3                      Last Accuracy     0.456       0.540
4                        Overall AUC       NaN       0.513
5                   Maximum F1 Score       NaN       0.441
6                       Minimum Loss       NaN       0.893
7  Difference in Average Loss Last N       NaN      -0.022
8  Standard Deviation of Loss Last N       NaN       0.049""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.626       0.592
1     Standard Deviation of Accuracy     0.243       0.198
2                      Best Accuracy     0.925       0.818
3                      Last Accuracy     0.925       0.788
4                        Overall AUC       NaN       0.539
5                   Maximum F1 Score       NaN       0.814
6                       Minimum Loss       NaN       0.683
7  Difference in Average Loss Last N       NaN       0.441
8  Standard Deviation of Loss Last N       NaN       0.091""",


"""                              Metric  Training  Validation
0                   Average Accuracy     0.348       0.341
1     Standard Deviation of Accuracy     0.019       0.025
2                      Best Accuracy     0.407       0.409
3                      Last Accuracy     0.363       0.369
4                        Overall AUC       NaN       0.491
5                   Maximum F1 Score       NaN       0.118
6                       Minimum Loss       NaN       1.070
7  Difference in Average Loss Last N       NaN      -0.003
8  Standard Deviation of Loss Last N       NaN       0.009""", 

"""                              Metric  Training  Validation
0                   Average Accuracy     0.779       0.678
1     Standard Deviation of Accuracy     0.178       0.111
2                      Best Accuracy     0.945       0.783
3                      Last Accuracy     0.942       0.773
4                        Overall AUC       NaN       0.530
5                   Maximum F1 Score       NaN       0.779
6                       Minimum Loss       NaN       0.747
7  Difference in Average Loss Last N       NaN       0.872
8  Standard Deviation of Loss Last N       NaN       0.156""",

"""                              Metric  Training  Validation
0                   Average Accuracy     0.407       0.409
1     Standard Deviation of Accuracy     0.082       0.101
2                      Best Accuracy     0.696       0.722
3                      Last Accuracy     0.406       0.404
4                        Overall AUC       NaN       0.508
5                   Maximum F1 Score       NaN       0.698
6                       Minimum Loss       NaN       0.723
7  Difference in Average Loss Last N       NaN      -0.012
8  Standard Deviation of Loss Last N       NaN       0.109"""



    # More tables would follow here
]

result = process_metric_tables(metric_tables)
print(result)


                           Metric       Training     Validation
                 Average Accuracy (0.507, 0.187) (0.483, 0.144)
   Standard Deviation of Accuracy (0.114, 0.094) (0.103, 0.063)
                    Best Accuracy (0.694, 0.244) (0.685, 0.162)
                    Last Accuracy (0.618, 0.290) (0.575, 0.198)
                      Overall AUC            NaN (0.516, 0.019)
                 Maximum F1 Score            NaN (0.570, 0.292)
                     Minimum Loss            NaN (0.823, 0.159)
Difference in Average Loss Last N            NaN (0.255, 0.397)
Standard Deviation of Loss Last N            NaN (0.083, 0.056)
